In [1]:
import numpy as np
import pandas as pd
import os
import time
import json
import gc
from pandas.io.json import json_normalize
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

### 第一步，我们先把json格式展开

In [ ]:
def load_df(csv_path, nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [ ]:
train = load_df('/users/lichanghao/data/R/train.csv')
gc.collect()

In [ ]:
train.to_hdf('/users/lichanghao/tempfile/train.h5','train')

In [2]:
train = pd.read_feather('/home/leechh/tempfile/R/train.feather')

In [3]:
train.head(3)

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device_browser,device_browserSize,...,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_campaignCode,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,None,None,None,(not set),None,None,(not provided),organic,None,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,None,None,None,(not set),None,None,(not provided),organic,None,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,None,None,None,(not set),None,None,(not provided),organic,None,google


### totals_transactionRevenue

首先呢，我们先看看我们要预测的transactionRevenue 大概是什么样，

In [ ]:
train['totals_transactionRevenue'] = train.totals_transactionRevenue.astype('float')

In [ ]:
def plotRevence():
    idx = train.totals_transactionRevenue.notnull()
    nozero = sum(idx)
    iszero = train.shape[0] - nozero
    plt.figure(figsize=[14,6])
    plt.subplot(1,2,1)
    plt.pie([nozero,iszero],
            labels=[f'{nozero} nozero',f'{iszero} iszero'],
            radius=1,
            colors=['salmon','lightgray'], 
            explode=[0.3,0])
    plt.title('transactionRevenue')

    plt.subplot(1,3,3)
    revenue = np.log1p(train[idx].totals_transactionRevenue.astype('float')+1)
    plt.hist(revenue, 
             bins=200, 
             color='lightgray')
    plt.plot([np.average(revenue),np.average(revenue)],[0,500],c='salmon',linestyle='--')
    plt.title('non-zero transactionRevenue')
    plt.show()
    return None


plotRevence()

In [ ]:
train['target'] = np.log1p(train.totals_transactionRevenue.fillna(0,inplace=False))

我们可以看到，可以说20/80法则发挥着决定性的作用，绝大多数的点击量都是没有成交的，只有大概百分之一的进入是会成交的，而在成交的金额上，在经过ln之后基本符合正态分布，这是个不错的开始。

### channelGrouping 

In [ ]:
def plotchannel():
    channel = train.channelGrouping.value_counts(dropna=True,normalize=True).sort_index()
    idx = train.totals_transactionRevenue.notnull()
    channel2 = train[idx].channelGrouping.value_counts(dropna=True,normalize=True).sort_index()
    channelmean = train.groupby('channelGrouping').mean().totals_transactionRevenue
    channelpercentage = train[idx].channelGrouping.value_counts(dropna=True,normalize=False)/ train.channelGrouping.value_counts(dropna=True,normalize=False)

    plt.figure(figsize=[10,15])
    plt.subplot(3,1,1)
    plt.bar(np.arange(channel.shape[0])+1,
            channel.values,
            width=np.ones(8)*0.2,
            facecolor='lightgray',
            edgecolor='lightgray',
            tick_label=channel.index.values,
            label='with zero data')
    plt.bar(np.arange(channel2.shape[0])+0.8,
            channel2.values,
            width=np.ones(8)*0.2,
            facecolor='gray',
            edgecolor='gray',
            tick_label=channel.index.values,
            label='without zero data')
    plt.title('channelGrouping with iszero data')
    plt.legend()

    plt.subplot(3,1,2)
    plt.bar(np.arange(channel2.shape[0])+1,
            np.log1p(channelmean.values+1),
            facecolor='gray',
            width=np.ones(8)*0.2,
            edgecolor='gray',
            tick_label=channelmean.index.values)
    plt.plot(np.arange(channel2.shape[0])+1,np.log1p(channelmean.values+1),'.--',c='salmon')
    plt.title('channelGrouping mean')
    plt.ylim([0,40])

    plt.subplot(3,1,3)
    plt.bar(np.arange(channelpercentage.shape[0])+1,
            channelpercentage.values,
            facecolor='gray',
            width=np.ones(8)*0.2,
            edgecolor='gray',
            tick_label=channelpercentage.index.values)
    plt.plot(np.arange(channel2.shape[0])+1,channelpercentage.values,'.--',c='salmon')
    plt.title('channelGrouping precentage')
    plt.ylim([0,0.07])
    plt.show()
    return None

plotchannel()

第一类：市场投放活动（Campaigns）
　　这一类比较常见的是付费关键词即Adwords，如果是Adwords，你不用加码，GA也能非常好的识别出来。当然如果你给自己的某个外部投放加上码，GA也会认为它是一个投放活动。投放活动在GA所有流量来源里，优先级别是最高的。

第二类：自然搜索（Organic）
　　这个大家很理解，就是Google和Baidu等自然搜索引入的流量。它对于GA来说，优先级别与Adwords是一样的。

第三类：推介网站（Referral）
　　这一渠道的流量主要是指从其他网站点过来的链接，比如友情链接等，这类来源通常是免费的。这一来源的优先级别低于市场投放活动和自然搜索。

第四类：直接输入（Direct）
　　直接输入包括几个方面，一个是直接在地址里输入网站，第二个是利用收藏夹与标签，第三个是从桌面软件，比如从QQ，MSN等软件点过来的链接，第四个是如果无法判别流量来源的话，也会归入直接输入。这个来源的优先级是最低的，大家抢的都是这个流量。

结论： 由上图我们可以看出，social项拥有很高的浏览量，但是成交比很低，很多人都只是单纯的点进来看看，并没有买任何东西。social项很可能指的是网站在社交媒体上投放广告。反之Referral项虽然没有太多的浏览量，但是成交比却很高。 当然，关键词广告自然搜索所占浏览量很大，但是成交比并不大。

### 我觉得seesionId 应该就是fullvisitId 与 visitId 的结合，验证以下，如果是的话，我们就可以把 这个features去掉了。

In [ ]:
print('fullvisitId sess diff:',sum(train.sessionId.str.split('_',expand=True)[0] != train.fullVisitorId))
print('visitId sess diff:',sum(train.sessionId.str.split('_',expand=True)[1] != train.visitId.astype('str')))

In [ ]:
train.drop(['sessionId'],axis=1,inplace=True)

### 接下来，把时间date转化为时间格式

In [4]:
train['date'] = pd.to_datetime(train.date,format='%Y%m%d')


In [ ]:
holiidx = (train.date > '2016-11-24') & (train.date < '2016-12-25')

In [ ]:


def plotdate():
    idx = train.totals_transactionRevenue.notnull()
    datetime1 = train[idx].date.value_counts(normalize=True).sort_index()
    datetime2 = train.date.value_counts(normalize=True).sort_index()
    
    date1 =  train[idx].date.value_counts().sort_index()
    date2 = train.date.value_counts().sort_index()
    
    tofirst = np.array([i.days for i in train[idx].date-datetime1.index.values[0]])
    plt.figure(figsize=[30,15])
    plt.subplot(3,1,1)
    plt.title(f'Daily turnover,from {datetime1.index.values[0]} to {datetime1.index.values[-1]}')
    plt.plot(np.arange(datetime1.shape[0])+1,
             datetime1.values,
             '.-',
             c='gray',
             label='no zero data')
    plt.plot(np.arange(datetime2.shape[0])+1,
             datetime2.values,
             '.-',
             c='salmon',label='have zero data')
    plt.grid()
    plt.legend()
    
    plt.subplot(3,1,2)
    plt.plot(np.arange(date2.shape[0]),
             (date1/date2).values,
             '.-',
             c='salmon',
             label='probability')
    plt.grid()
    plt.legend()
    
    plt.subplot(3,1,3)
    plt.scatter(tofirst,np.log1p(train[idx].totals_transactionRevenue+1),s=0.5,alpha=1,c='gray')
    plt.ylim([12,22])
    plt.show()
    return None


plotdate()

从这个图我们可以看出一些特别有趣的现象，首先浏览量是以一周为周期变化的，所以我们应该创建一个UTC星期特征。但是，由下图可知，时间与金额无关。

In [ ]:
train['week'] = train.date.apply(lambda t: t.dayofweek)

我们创建一个新的特征 holiday，可以标记处是否处于感恩节以及圣诞节假期

In [ ]:
pd.Timestamp('2016-12-25')-pd.Timestamp('2016-8-1')

In [ ]:
idx = train.trafficSource_adContent == 'Google Merchandise Collection'

date1 =  train[idx].date.value_counts().sort_index()
date2 = train.date.value_counts().sort_index()
plt.figure(figsize=[15,3])
plt.plot(np.arange(date2.shape[0]),
         (date1/date2).values,
         '.-',
         c='salmon',
         label='probability')
plt.grid()
plt.legend()


题外话：这个图解释了广告是怎么影响销售量的，在第200天左右投放的一条脍炙人口的广告，与销售量几乎同时上涨。所以我们可以创建一个新的特征，googAD，这个特征指的是一个很火的广告存在的时期。

In [ ]:
idx = train.trafficSource_adContent == 'Google Merchandise Collection'
holi = train[idx].date.apply(lambda t: t.dayofyear).values
train['goodAD'] = train.date.apply(lambda t: t.dayofyear in holi)

### socialEngagementType

In [ ]:
idx = (train.socialEngagementType != 'Not Socially Engaged')
sum(idx)

In [ ]:
train.drop(['socialEngagementType'], axis=1, inplace=True)

In [ ]:
train.head()

### visitNumber

In [ ]:
def plotvisitnumber():
    idx = (train.totals_transactionRevenue.notnull())
    vn1 = train.visitNumber.value_counts(dropna=False)[0:20]
    vn2 = train[idx].visitNumber.value_counts(dropna=False)[0:20]
    plt.figure(figsize=[20,7])
    plt.subplot(1,2,1)
    plt.bar(np.arange(vn1.shape[0])+1,
            vn1.values,
            facecolor='lightgray',
            width=np.ones(20)*0.25,
            edgecolor='lightgray',
            tick_label=vn1.index.values)
    plt.plot(np.arange(vn1.shape[0])+1,
             vn1.values,'.--', 
             c='salmon')
    plt.title('visitNumber with zero data')
    plt.grid()
    plt.subplot(1,2,2)
    plt.bar(np.arange(vn2.shape[0])+1,
            vn2.values,
            facecolor='lightgray',
            width=np.ones(20)*0.25,
            edgecolor='lightgray',
            tick_label=vn2.index.values)
    plt.plot(np.arange(vn2.shape[0])+1,
             vn2.values,'.--', 
             c='salmon')
    plt.title('visitNumber without zero data')
    plt.grid()
    plt.show()
    return None


plotvisitnumber()

In [ ]:
plt.scatter(train.visitNumber,train.totals_transactionRevenue/10**(6),s=0.5,alpha=0.5,c='salmon')
plt.xlim([0,50])

我们可以看到，visitNumber及其不平衡，大多数为1，由kaggle官方提供的说明，If this is the first session, then this is set to 1，所以大多数都是用户初次访问,但是对于成交的浏览来说，曲线更平缓一些。但是成交金额与浏览次数没有线性关系

### visitStartTime
The timestamp (expressed as POSIX time)， so we convert it to a pd.datatime

In [ ]:
def totime(t):
    year = time.localtime(t)[0]
    month = time.localtime(t)[1]
    day = time.localtime(t)[2]
    hour = time.localtime(t)[3]
    minute = time.localtime(t)[4]
    second = time.localtime(t)[5]
    return pd.Timestamp(year,month,day,hour,minute,second)


train['visitStartTime'] = train.visitStartTime.map(totime)

这个features, 因为使用的是UTC，所以在时间上面有一定偏移，在之后的工作中，如果我们能否通过所在地区找到其时区，便能探索具体时间对于结果的影响。

### 去掉重复features

In [ ]:
for i in train.columns:
    if sum(train[i] == train[i][0]) == train.shape[0]:
        train.drop([i], axis=1, inplace=True)
print(f'After drop feature Number is: {train.shape[1]}')

### device

In [ ]:
train.iloc[:,6:10].head()

In [ ]:
# 我们去掉device_isMobile
train.drop(['device_isMobile'], axis=1, inplace=True)

我们可以看到device_operatingSystem 有较多(not set)项目 

In [ ]:
def plotdevice(df,top=10):
    n = 1
    plt.figure(figsize=[20,10])
    for i in df.columns[6:9]:
        idx = train.totals_transactionRevenue.notnull()
        value_count1 = train[i].value_counts(dropna=True,normalize=True)[0:top]
        value_count2 = train[idx][i].value_counts(dropna=True,normalize=True)[0:top]
        plt.subplot(2,3,n)
        plt.bar(np.arange(value_count1.shape[0])+1,
                value_count1.values,           
                width=np.ones(value_count1.shape[0])*0.2,
                facecolor='lightgray',
                edgecolor='lightgray',
                tick_label=value_count1.index.values,
                label='have zero data')
        plt.legend()
        plt.title(f'{i},{sum(train[i].isnull())} NaN data')
        plt.grid()
        plt.subplot(2,3,n+3)
        plt.grid()
        plt.bar(np.arange(value_count2.shape[0])+1.2,
                value_count2.values,           
                width=np.ones(value_count2.shape[0])*0.2,
                facecolor='gray',
                edgecolor='gray',
                tick_label=value_count2.index.values,
                label='no zero data')
        plt.legend()
        n = n+1
    plt.show()
    return None


plotdevice(train,top=5)

In [ ]:
idx = train.device_browser.value_counts().index[10:-1].values
train.device_browser.replace(idx,'other').value_counts()

In [ ]:
train.device_browser.unique()

我们可以看到，虽然chrome浏览器拥有较高的市场占有率，但是chrome用户也拥有更高的购买热情，可能都是google的死忠？ps:I use firefox.: )   
以及大家用桌面机的时候更倾向于交易，可能是交易需要信用卡等等信息，比较麻烦，或者单纯因为app做的不好？有趣的是mac用户拥有更高的交易意愿，可能是因为Mac比较贵？


### geoNetwork

In [ ]:
train.iloc[:,9:16].head()

In [ ]:
def plotgeonetwork(df,top=10):
    n = 1
    plt.figure(figsize=[20,40])
    for i in df.columns[9:16]:
        idx = df.totals_transactionRevenue.notnull()
        value_count1 = df[i].value_counts(dropna=True,normalize=True)[0:top]
        value_count2 = df[idx][i].value_counts(dropna=True,normalize=True)[0:top]
        plt.subplot(7,2,n)
        plt.bar(np.arange(value_count1.shape[0])+1,
                value_count1.values,           
                width=np.ones(value_count1.shape[0])*0.2,
                facecolor='lightgray',
                edgecolor='lightgray',
                tick_label=value_count1.index.values,
                label='have zero data')
        plt.legend()
        plt.title(f'{i},unique is: {df[i].nunique()},num is: {sum(df[i].notnull())}')
        plt.grid()
        plt.subplot(7,2,n+1)
        plt.grid()
        plt.bar(np.arange(value_count2.shape[0])+1.2,
                value_count2.values,           
                width=np.ones(value_count2.shape[0])*0.2,
                facecolor='gray',
                edgecolor='gray',
                tick_label=value_count2.index.values,
                label='no zero data')
        plt.title(f'{i},unique is: {df[idx][i].nunique()},num is: {sum(df[idx][i].notnull())}')
        plt.legend()
        n = n+2
    plt.show()
    return None


plotgeonetwork(train,top=5)

我们似乎可以看到，尽管全球都有浏览量，但是貌似绝大多数都是美国人在购买，并且这几个features都存在大量诸如not set，not available in demo dataset 之类的变量，之后我们要看看造成这种现象的原因。 

In [ ]:
index =  train.geoNetwork_city == 'not available in demo dataset'
plotgeonetwork(train[index],top=5)

啊哈，一个有趣的发现，那就是city中的的not available in demo dataset其实是在city，metro,region都是not available in demo dataset的。而在其他几项中并没有明显不同

In [ ]:
train['is not available'] = train.geoNetwork_city == 'not available in demo dataset'

In [ ]:
idx = train['is not available'] == True
idx2 = train.totals_transactionRevenue.notnull()
print(sum(idx),sum(idx & idx2))

创建一个特征 domain，这个特征是由于 geoNetwork_networkDomain 中的  'unknown.unknown' 在正反例中波动过大，所以针对是否是 'unknown.unknown' ，是否是(not set) 设立成一个特征。然后就可以删除掉 geoNetwork_networkDomain.     
创建特征 notsetcity 与 notsetmetro ，因为这两个特征为 True 时，大概率属于亚洲地区，购买的意愿会大幅度降低，可能是因为运费或者价格太贵。
创建特征 isUS ，当国家为 US 时为True。

### 建议：
删除掉'geoNetwork_city'，'geoNetwork_region'，'geoNetwork_metro'，'geoNetwork_country'

### totals

In [ ]:
train.iloc[:,16:20].head()

In [ ]:
train['totals_bounces'] = train.totals_bounces.fillna(0)
train['totals_newVisits'] = train.totals_newVisits.fillna(0)


In [ ]:
train.iloc[:,16:20].head()

In [ ]:
def plottotals(df,top=5):
    n = 1
    plt.figure(figsize=[10,20])
    for i in df.columns[16:20]:
        idx = df.totals_transactionRevenue.notnull()
        value_count1 = df[i].value_counts(dropna=True,normalize=True)[0:top]
        value_count2 = df[idx][i].value_counts(dropna=True,normalize=True)[0:top]
        plt.subplot(4,2,n)
        plt.bar(np.arange(value_count1.shape[0])+1,
                value_count1.values,           
                width=np.ones(value_count1.shape[0])*0.2,
                facecolor='lightgray',
                edgecolor='lightgray',
                tick_label=value_count1.index.values,
                label='have zero data')
        plt.legend()
        plt.title(f'{i},unique is: {df[i].nunique()}')
        plt.grid()
        plt.subplot(4,2,n+1)
        plt.grid()
        plt.bar(np.arange(value_count2.shape[0])+1.2,
                value_count2.values,           
                width=np.ones(value_count2.shape[0])*0.2,
                facecolor='gray',
                edgecolor='gray',
                tick_label=value_count2.index.values,
                label='no zero data')
        plt.title(f'{i},unique is: {df[idx][i].nunique()}')
        plt.legend()
        n = n+2
    plt.show()
    return None


plottotals(train,top=20)

emmmm~,一个了不起的发现，最终预测结果是否为0与totals_bounces相关，如果bounces=1则交易必然失败，但如果为0，也不一定成功。并且如果点击量与网页浏览量在10到30之间，则很有可能交易成功，并且我们还可以看到，老顾客购物意愿比从其他途径来的新客户更强烈。

In [ ]:
train['totals_bounces'] = train.totals_bounces.astype('float')
train['totals_hits'] = train.totals_hits.astype('float')
train['totals_newVisits'] = train.totals_newVisits.astype('float')
train['totals_pageviews'] = train.totals_pageviews.astype('float')

In [ ]:
def scattertotals(df):
    idx = df.totals_transactionRevenue.notnull()
    df['totals_hits'] = df.totals_hits.astype('float')
    hits = df[idx].groupby('totals_hits').mean().target
    newvisit = df[idx].groupby('totals_newVisits').mean().target
    pageviews = df[idx].groupby('totals_pageviews').mean().target
    
    plt.figure(figsize=[10,15])
    plt.subplot(3,1,1)
    plt.bar(hits.index.values,hits.values,alpha=0.1,facecolor='k')
    plt.scatter(df.totals_hits.values,df.target.values,s=0.1,c='salmon')
    plt.ylim([14,22])
    plt.xlim([0,100])
    return None

scattertotals(train)

### trafficSource

In [ ]:
train.iloc[:,21:34].head()

In [ ]:
train.drop(['trafficSource_campaignCode'],axis=1,inplace=True)

## 探索Nan的分布

In [ ]:
col2 = ['trafficSource_adContent',
       'trafficSource_adwordsClickInfo.adNetworkType',
       'trafficSource_adwordsClickInfo.gclId',
       'trafficSource_isTrueDirect', 
        'trafficSource_keyword',
        'trafficSource_referralPath']

col = ['adContent',
       'adNetworkType',
       'gclId',
       'isTrueDirect', 
       'keyword',
       'referralPath']

num = []
for i in col2:
    for j in col2:
        idx = train[i].notnull() & train[j].notnull()
        num.append(sum(idx))
data = np.array(num).reshape(6,6)

df = pd.DataFrame(data=data,columns=col,index=col)
df

### 结论：我对上述几项进行了两两对比，并且与channelgroup等项进行了对比，发现：      
1. referralPath项，只有当medium为referral时，才有值      
2. campaignCode：11251kjhkvahf    
3. adContent项是google广告中出现的词，将近一半与ClickInfo类，campaign，gclId有交集，但是是哪一半？啊，我们找到了一些，如果广告词中出现了'google'那么这两项的值就是共同存在的      
4. Clickinfo 类与campaign，gclId的值域基本重合，并且每一个都有关键词   
5. gclId项是google 点击 ID.   
6. 如果您服务器上的网址重写引擎更改了自动标记 gclid 值的大小写，如将 gclid=TeSter 改写为全部小写的 gclid=tester，则 Google Analytics（分析）会无法确定哪次 Google Ads 点击与哪个会话相关。如果存在任何 gclid 参数，那么系统会将数据来源识别为 source=google 和 medium=cpc，但所有其他特定于点击的数据（例如广告系列、广告组、关键字等）将在 Google Analytics（分析）报告中显示为 (not set)。   
7. gclid 与 channelGroup 中的Paid Search 高度重合，所有data如果存在gclid 那么必然来源于 付费搜索，这也就解释了为什么referralPath不与其他项相交，因为referralPath 必然来源于 referral和social。

## 探索train.trafficSource值之间的关系

In [ ]:
train.columns[21:33]

In [ ]:
idx1 = train.channelGrouping != 'Direct'
idx2 =  train.trafficSource_isTrueDirect.notnull()
idx = idx1 & idx2
print(sum(idx),sum(idx1),sum(idx2),sum(train.channelGrouping=='Referral'))
print(train.channelGrouping.value_counts())
print(train.trafficSource_medium.value_counts())

In [ ]:
idx = train.trafficSource_isTrueDirect.notnull()
train[idx].channelGrouping.value_counts()

### 结论：
1. medium项与channelgroup高度重合，只是medium中的referral在Organic Search中被拆分成了Social与Direct，所以建议删除trafficSource_medium
2. 可以删除trafficSource_adwordsClickInfo.isVideoAd，因为没有意义
3. 可以删除trafficSource_adwordsClickInfo.adNetworkType，没啥意义
4. 可以删除trafficSource_adwordsClickInfo.page 和 trafficSource_adwordsClickInfo.slot ，也没啥意义
5. 建议删除trafficSource_keyword，trafficSource_campaign，还是没啥意义

### 所以我们剩下什么了？
1. 'trafficSource_adContent'：只有cahnnelgrouping 为Paid Search时才有值，并且出项google时便会拥有一个gclid   
2. 'trafficSource_adwordsClickInfo.gclId'：我还没想出怎么去利用这一串乱码，并且只有cahnnelgrouping 为Paid Search时才有值，并且一个fullVisitorId只对应一个gclid,其实吧，可以吧是否存在gclid作为一个feature，然后把这个删掉。。。
#### important! 
3. 'trafficSource_isTrueDirect'：如果会话的来源是“Direct”（表示用户在浏览器中输入了您网站网址的名称或是通过书签访问了您的网站），则为 True；如果 2 个连续但不同的会话具有完全相同的广告系列详细信息，则此字段也为 true。否则为 NULL。所以，我们可以把这两种情况区分开。
4. 'trafficSource_referralPath' ：如果 trafficSource.medium 为“referral”，则此值为引荐来源网址的路径。（引荐来源网址的主机名包含在 trafficSource.source 中。）只有cahnnelgrouping 为 referral 和 Social 的这一项才有值，我们似乎可以从这里也找到一些出国别之类或者习惯之类的信息。  
5. 'trafficSource_source'：流量来源。可以是搜索引擎的名称、引荐来源网址的主机名或 utm_source 网址参数的一个值。

In [ ]:
idx = (train.trafficSource_isTrueDirect.notnull()) & (train.channelGrouping != 'Direct') 
idx2 = train.totals_transactionRevenue.notnull()
print(train[idx].channelGrouping.value_counts())
print(sum(idx),sum(idx & idx2))

### 新的特征：
我们现在可以创建一个新的特征，当istrueDirect不为空，channelgrouping不为direct，totoals_bounces为0，创建新特征 isTrueDirect ，这个特征貌似有非常好的效果。
并且，我们可以把trafficSource_source 和 trafficSource_referralPath 拼接起来。组成一个新的特征 path。

In [ ]:
train['path'] = train.trafficSource_source +  train.trafficSource_referralPath.fillna('') 

In [ ]:
idx = train.totals_transactionRevenue.notnull()

for i in train.columns[21:33]:
    print('\n',
          train[i].value_counts(),
          '\n',
          'len is: ', train[i].value_counts().shape[0],
          'Num is: ',sum(train[i].notnull()),
          'Nan is: ',sum(train[i].isnull()),
          '\n',
          '-'*80,
          '\n',
          train[idx][i].value_counts(),      
          '\n',
          'len is: ',train[idx][i].value_counts().shape[0],
          'Num is: ',sum(train[idx][i].notnull()),
          'Nan is: ',sum(train[idx][i].isnull()),
          '\n',
          '*'*80)

In [ ]:
idx = train['trafficSource_adwordsClickInfo.slot'] == 'RHS'
train[idx].date

In [ ]:
idx = train.totals_transactionRevenue.notnull()

print(
    train.path.value_counts()[0:10],
    'Num is: ',sum(train[i].notnull()),
    '\n','*'*40,'\n',
    train[idx].path.value_counts()[0:10],
    'Num is: ',sum(train[idx][i].notnull()),
)

### 建议：
去掉 trafficSource_source 和 trafficSource_referralPath 

In [5]:
adcontentidx = train.trafficSource_adContent == 'Google Merchandise Collection'
train['day'] = train.date.apply(lambda t: t.dayofyear)
num = train[adcontentidx].day.value_counts()

In [6]:
def isgooglemc(day,numb):
    if day in numb.index.values:
        val = numb[day]
    else:
        val = 0
    return val

In [7]:
a = train.day.apply(isgooglemc,args=[num])

In [16]:
a[a!=0].shape

(382600,)